In [1]:
from pulp import *

In [2]:
import json
import numpy as np

In [3]:
from cancer_nutrition.utils.db.frontend import DBProvider
from cancer_nutrition.utils.constants.db import *
from cancer_nutrition.utils.form.form import form_to_dict

In [4]:
fpath = "/Users/andreyignatov/Downloads/test-1689066441.0527298.json"
# fpath = "/Users/andreyignatov/Downloads/test-1689066441.0527298.json"

with open(fpath) as f:
    form_dict_str = f.read()
    form_dict_str = form_dict_str.replace("ImmutableMultiDict(", "")[:-1]
    form_dict_str = form_dict_str.replace("(", "[").replace(")", "]").replace("'", "\"")
    
    form_dict = {k: v for (k,v) in json.loads(form_dict_str)}

In [5]:
db_provider = DBProvider(DB_NAME, DB_USER, DB_PASSWD, DB_HOST, 11459)

In [140]:
# data_config = db_provider.get_data_config()

In [141]:
data_config = np.load("./data_config.npy", allow_pickle=True)
# data_config = 

data_config = data_config.item()

In [142]:
from cancer_nutrition.utils.value import str_is_unit, Value, Analysis
from datetime import date

# 1. Getting patient info

In [143]:
from cancer_nutrition.patient import PatientInfo

In [144]:
form_dict["1^7"] = "test"
form_dict["1^8"] = "test"


In [145]:
form_patient_info = form_to_dict(form_dict, data_config)

In [146]:
from cancer_nutrition.utils.age import Age

In [147]:
patient_info = PatientInfo(form_patient_info, db_provider)

1it [00:00,  1.20it/s]


In [149]:
{k: patient_info.data[0][k] for k in patient_info.data[0]["Производные параметры"]}

{'Возраст': <cancer_nutrition.utils.age.Age at 0x12a1d8af0>,
 'ИМТ': 20.25462962962963,
 'z-score роста': 0.9039164463919938,
 'z-score массы тела': None,
 'z-score ИМТ': 1.632050097270929,
 'Нормальный нутритивный статус': False,
 'Легкая белково-энергетическая недостаточность': False,
 'Умеренная белково-энергетическая недостаточность': False,
 'Тяжелая белково-энергетическая недостаточность': False,
 'Избыточная масса тела': True,
 'Ожирение 1 степени': False,
 'Ожирение 2 степени': False,
 'Ожирение 3 степени': False,
 'Ожирение тяжелое': False,
 'Дефицит висцерального пула белка': True,
 'Тек*Небелковые ккал': 22.352941176470587,
 'СКФ': 116.82026932668329,
 'Острая печеночная недостаточность': False,
 'Лабораторные признаки холестаза': 0.0,
 'Острое повреждение почек': 0,
 'Лабораторные признаки острого панкреатита': 0,
 'Ожидаемая токсичность': 0,
 'Идеальная масса тела': None}

In [150]:
def _make_problem(Ps, Ls, Cs, Es, f,
                  enterals,
                  enteral_tags,
                  tag_weights,
                  m, age_years,
                  sipping=True,
                  disabled=None):
    """
    Ps, Ls, Cs, Es - посчитанные по формулам белок, жиры, углеводы, ккал
    f - доля питания
    m - масса, кг
    """
    M = 1e7

    prob = LpProblem("en-selection", LpMinimize)

    P = LpVariable("P")
    L = LpVariable("L")
    C = LpVariable("C")
    E = LpVariable("E")

    en_weights = {k: sum([(tag in en_tags) * weight
                          for tag, weight in tag_weights.items()])
                  for k, en_tags in enteral_tags.items()}
    
    # i'th enteral chosen or not
    xs = LpVariable.dicts("x",
                          (i for i in range(len(enterals))),
                          cat="Binary")
    ent_doses = LpVariable.dicts("xv",
                                 (i for i in range(len(enterals))),
                                 cat="Continuous",
                                 lowBound=0)

    f1, f2 = LpVariable("abs_plc_delta"), LpVariable("n_nutritions")
    f3, f4 = LpVariable("cvk_selection"), LpVariable("total_volume")
    f5, f6 = LpVariable("tagging"), LpVariable("sipping_tube")

    if disabled is None:
        disabled = []
    for i, (name, features) in enumerate(enterals.items()):
        if "Углеводы/100мл" not in features:
            disabled += [i]

    prob += P * f == lpSum([features["Белок/100мл"] * ent_doses[i]
                            for i, (name, features) in enumerate(enterals.items()) if i not in disabled])
    prob += L * f == lpSum([features["Жиры/100мл"] * ent_doses[i]
                            for i, (name, features) in enumerate(enterals.items()) if i not in disabled])
    prob += C * f == lpSum([features["Углеводы/100мл"] * ent_doses[i]
                            for i, (name, features) in enumerate(enterals.items()) if i not in disabled])
    prob += E * f == lpSum([features["ККал/100мл"] * ent_doses[i]
                            for i, (name, features) in enumerate(enterals.items()) if i not in disabled])

    for i in range(len(enterals)):
        prob += ent_doses[i] <= xs[i] * M

    for i, (par_name, par) in enumerate(enterals.items()):
        if ("Возраст" not in par) or (age_years < par["Возраст"]):
            prob += xs[i] == 0
        if disabled is not None and par_name in disabled:
            prob += xs[i] == 0

    mp, ml, mc, me = LpVariable("mp"), LpVariable("ml"), LpVariable("mc"), LpVariable("me")

    prob += mp >= P - Ps
    prob += mp >= Ps - P
    prob += ml >= L - Ls
    prob += ml >= Ls - L
    prob += mc >= C - Cs
    prob += mc >= Cs - C
    prob += me >= E - Es
    prob += me >= Es - E

    prob += lpSum([ent_doses[i] for i in range(len(enterals))]) >= 1

    prob += f1 == mp + ml + mc
    prob += f2 == me
    prob += f4 == lpSum([ent_doses[i] for i in range(len(enterals))])
    prob += f5 == -lpSum([en_weights[k] * xs[i]
                          for i, k in enumerate(list(enterals.keys()))])
    prob += f6 == -sipping * lpSum([xs[i] * ("Сипинг" in v)
                                    for i, (k,v) in enumerate(enteral_tags.items())]) - (1 - sipping) * lpSum([xs[i] * ("Зонд/г-стома" in v)
                                                                                                               for i, (k,v) in enumerate(enteral_tags.items())])

    # MARK: no more than 3 items selected!
    prob += lpSum([xs[i] for i in range(len(enterals))]) <= 1

    c1, c2, c3, c4, c5, c6 = 1, 10, 0, 1, 10, 10
    prob += c1 * f1 + c2 * f2 + c4 * f4 + c5 * f5 + c6 * f6   # c3 * f3 +
    # prob += lpSum([xs[i] for i in range(len(enterals))])
    
    return prob

In [151]:
from cancer_nutrition.nutrition.main import NutritionRecommender, ProteinDerivation, CarbohydrateDerivation, LipidDerivation, EnergyDerivation


In [152]:
data_dict = patient_info.data[0]
enterals = data_dict["ЭП"]

In [153]:
from cancer_nutrition.utils.measurement.enteral_tagging import EnteralTagging
from cancer_nutrition.nutrition.enteral_selection import EnteralSelector

In [154]:
classes = {"Белок": ProteinDerivation,
           "Жиры": LipidDerivation,
           "Углеводы": CarbohydrateDerivation,
           "ККал": EnergyDerivation}
due_demands = {k + "_due": _class.compute(data_dict, due_only=True)
               for k, _class in classes.items()}
patient_info.data[0]["Риск рефидинг-синдрома"] = 0 # RefeedingRisk.compute(**patient_info.data[0], **due_demands)
# data_dict["Производные параметры"] = data_dict["Производные параметры"] + [RefeedingRisk.name]

# demands = {k: dict(zip(["value", "limits", "due"],
#                        _class.compute(data_dict)))
#            for k, _class in classes.items()}

# print(demands, due_demands)

# fractions = NutritionTypeFractions.compute(demands, **data_dict)
# data_dict["fractions"] = fractions

data_dict[EnteralTagging.name] = EnteralTagging.compute(**data_dict)

# out_demands, out_parenterals = ParenteralSelector.compute(data_dict, demands, data_dict["ПЭП"])
# print(out_parenterals, out_demands)

enteral_tags = db_provider.get_nutrition_tags(enteral=True)
out_en_demands, out_enterals = {}, {}

en_due_demands = {"ККал": due_demands["ККал_due"],
                  "Белок": EnteralSelector._p_en_f(data_dict["Возраст"]["лет"],
                                                   due_demands["ККал_due"]),
                  "Жиры": EnteralSelector._l_en_f(data_dict["Возраст"]["лет"],
                                                  due_demands["ККал_due"]),
                  "Углеводы": EnteralSelector._c_en_f(data_dict["Возраст"]["лет"],
                                                      due_demands["ККал_due"])
                  }

In [155]:
en_due_demands

{'ККал': 1953.7952000000002,
 'Белок': 50.310226400000005,
 'Жиры': 94.86761137777779,
 'Углеводы': 224.19799920000003}

In [156]:
prob = _make_problem(en_due_demands["Белок"], 
                     en_due_demands["Жиры"], 
                     en_due_demands["Углеводы"], 
                     en_due_demands["ККал"], 
                     1, data_dict["ЭП"], 
                     enteral_tags, data_dict[EnteralTagging.name], 
                     data_dict["Масса тела"]["кг"], data_dict["Возраст"]["лет"])

In [157]:
prob.solve()

1

In [158]:
def get_solution(lp_prob):
    prob_vars = lp_prob.variablesDict()
    out_demands = {key: prob_vars[key].value()
                   for key in ["P", "L", "C", "E"]}
    out_enterals = {}
    for i in range(len(enterals)):
        if prob_vars[f"x_{i}"].value() == 1 and prob_vars[f"xv_{i}"].value() > 0:
            val = prob_vars[f'xv_{i}'].value() * 100
            val = round(val / 10) * 10
            out_enterals[list(enterals.keys())[i]] = val

    return out_demands, out_enterals

In [159]:
get_solution(prob)

({'P': 38.270215, 'L': 93.661316, 'C': 223.57863, 'E': 1953.7952},
 {'Кальшейк (шоколад)': 1010})

In [160]:
data_dict[EnteralTagging.name]

{'Полимерная': 0.45,
 'Олигомерная': 0.05,
 'Гепа': 0.025,
 'Нефро': 0.025,
 'Пульмо': 0.025,
 'Диабет': 0.025,
 'Стандартная': 0.225,
 'Изокалорическая': 0.0625,
 'Гипокалорическая': 0.0125,
 'Гиперкалорическая': 0.05,
 'Содержит пробиотики': 0.0,
 'Без пробиотиков': 0.0625,
 'Содержит пищевые волокна': 0.015625,
 'Без пищевых волокон': 0.015625,
 'Содержит лактозу': 0.0078125,
 'Без лактозы': 0.0078125,
 'no_sip': 0.0,
 'no_en': 0.0}

In [161]:
_en_weights = {k: sum([(tag in en_tags) * weight
                          for tag, weight in data_dict[EnteralTagging.name].items()])
                  for k, en_tags in enteral_tags.items()}
    

In [163]:
data_dict["Производные параметры"]

['Возраст',
 'ИМТ',
 'z-score роста',
 'z-score массы тела',
 'z-score ИМТ',
 'Нормальный нутритивный статус',
 'Легкая белково-энергетическая недостаточность',
 'Умеренная белково-энергетическая недостаточность',
 'Тяжелая белково-энергетическая недостаточность',
 'Избыточная масса тела',
 'Ожирение 1 степени',
 'Ожирение 2 степени',
 'Ожирение 3 степени',
 'Ожирение тяжелое',
 'Дефицит висцерального пула белка',
 'Тек*Небелковые ккал',
 'СКФ',
 'Острая печеночная недостаточность',
 'Лабораторные признаки холестаза',
 'Острое повреждение почек',
 'Лабораторные признаки острого панкреатита',
 'Ожидаемая токсичность',
 'Идеальная масса тела']

In [162]:
_en_weights

{'Глутамин Плюс с апельсиновым вкусом': 0.36562500000000003,
 'Глутамин Плюс с нейтральным вкусом': 0.36562500000000003,
 'Дибен': 0.6156250000000001,
 'Изосурс': 0.815625,
 'Изосурс Протеин': 0.8031250000000001,
 'Изосурс Энерджи Ликвид': 0.8031250000000001,
 'Изосурс Энерджи Файбер': 0.8031250000000001,
 'Импакт Орал (ваниль)': 0.7406250000000001,
 'Импакт Орал (кофе)': 0.7406250000000001,
 'Импакт Орал (тропические фрукты)': 0.7406250000000001,
 'Импакт Энтерал': 0.815625,
 'Интестамин': 0.765625,
 'Кальшейк (ваниль)': 0.8109375000000001,
 'Кальшейк (клубника)': 0.8109375000000001,
 'Кальшейк (нейтральный)': 0.8109375000000001,
 'Кальшейк (шоколад)': 0.8109375000000001,
 'Кальшейк [банан]': 0.8109375000000001,
 'Кубизон': 0.8234375,
 'Малютка 1': 0.8234375,
 'Новасурс GI control': 0.8234375,
 'Новасурс Диабет Плюс': 0.603125,
 'Нутридринк (банан)': 0.8109375000000001,
 'Нутридринк (шоколад)': 0.8109375000000001,
 'Нутридринк [ваниль]': 0.8109375000000001,
 'Нутридринк [клубника]': 0